In [1]:
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.static import players
import pandas as pd

# Imported Libraries and NBA API
### Import required NBA API modules and core Python libraries for data extraction and processing.

In [2]:
import pandas as pd
from nba_api.stats.endpoints import leaguegamefinder

def get_finals_game_ids_for_player(player_id: int, season: str) -> list[str]:
    """
    Returns Finals game IDs for a player in a given season (e.g., '2016-17').
    Finals games have GAME_ID containing '040' in the series code.
    """
    gf = leaguegamefinder.LeagueGameFinder(
        player_id_nullable=player_id,
        season_nullable=season,
        season_type_nullable="Playoffs"
    )
    games = gf.get_data_frames()[0]  # game log

    # Finals filter: Finals series games have '040' in the GAME_ID (e.g., 0041600401)
    games["GAME_ID"] = games["GAME_ID"].astype(str)
    finals_games = games[games["GAME_ID"].str.contains("040")].copy()

    return finals_games["GAME_ID"].tolist()


# Player Identification 
### Identify the unique NBA player ID needed to query shot-level data from the NBA Stats API.

In [3]:
from nba_api.stats.endpoints import shotchartdetail

def get_player_shots_for_game(player_id: int, season: str, game_id: str) -> pd.DataFrame:
    scd = shotchartdetail.ShotChartDetail(
        team_id=0,
        player_id=player_id,
        season_nullable=season,
        season_type_all_star="Playoffs",
        game_id_nullable=game_id,
        context_measure_simple="FGA"
    )
    df = scd.get_data_frames()[0]
    return df


# Finals Game Identification 
### Retrieve all NBA Finals game IDs for a given player and season by filtering playoff games to the Finals round.

In [4]:
import time

def finals_shots_last_10_seasons(player_name: str, start_year=2014, end_year=2024):
    """
    Pulls ALL shots taken by player in ALL NBA Finals games from seasons start_year-end_year.
    Seasons are strings like '2016-17'. end_year refers to the ending year of the season.
    """
    from nba_api.stats.static import players

    # find player_id
    p = players.find_players_by_full_name(player_name)[0]
    player_id = p["id"]

    all_shots = []
    summary = []

    for end in range(start_year+1, end_year+1):  # 2016..2024 ends
        season = f"{end-1}-{str(end)[-2:]}"      # '2015-16' style

        finals_game_ids = get_finals_game_ids_for_player(player_id, season)
        summary.append({"season": season, "finals_games_found": len(finals_game_ids)})

        for gid in finals_game_ids:
            df_game = get_player_shots_for_game(player_id, season, gid)
            if not df_game.empty:
                all_shots.append(df_game)
            time.sleep(0.8)  # slow down to avoid blocking

    shots_df = pd.concat(all_shots, ignore_index=True) if all_shots else pd.DataFrame()
    summary_df = pd.DataFrame(summary)

    return shots_df, summary_df

# Shot Data Extraction (Per Game)

# Star Players 
### (LeBron James, Stephen Curry, Giannis Antetokounmpo, Nikola Jokic, Jimmy Butler)

# LeBron James - Cleveland Caveliers and Los Angeles Lakers (2015, 2016, 2017, 2018, 2020)

In [6]:
shots_df, finals_summary = finals_shots_last_10_seasons("LeBron James", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2014-15                   6
1  2015-16                   7
2  2016-17                   5
3  2017-18                   4
4  2018-19                   0
5  2019-20                   6
6  2020-21                   0
7  2021-22                   0
8  2022-23                   0
9  2023-24                   0
TOTAL SHOTS: 687


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0041400406,16,2544,LeBron James,1610612739,Cleveland Cavaliers,1,10,0,...,Left Side(L),24+ ft.,24,-228,77,1,0,20150616,CLE,GSW
1,Shot Chart Detail,0041400406,19,2544,LeBron James,1610612739,Cleveland Cavaliers,1,9,31,...,Center(C),Less Than 8 ft.,0,2,6,1,1,20150616,CLE,GSW
2,Shot Chart Detail,0041400406,57,2544,LeBron James,1610612739,Cleveland Cavaliers,1,4,50,...,Center(C),Less Than 8 ft.,0,6,0,1,0,20150616,CLE,GSW
3,Shot Chart Detail,0041400406,76,2544,LeBron James,1610612739,Cleveland Cavaliers,1,2,43,...,Center(C),Less Than 8 ft.,1,-11,7,1,0,20150616,CLE,GSW
4,Shot Chart Detail,0041400406,79,2544,LeBron James,1610612739,Cleveland Cavaliers,1,2,20,...,Center(C),Less Than 8 ft.,0,9,-3,1,1,20150616,CLE,GSW


# Stephen Curry - Golden State Warriors (2015, 2016, 2017, 2018, 2019, 2022)

In [7]:
shots_df, finals_summary = finals_shots_last_10_seasons("Stephen Curry", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2014-15                   6
1  2015-16                   7
2  2016-17                   5
3  2017-18                   4
4  2018-19                   6
5  2019-20                   0
6  2020-21                   0
7  2021-22                   6
8  2022-23                   0
9  2023-24                   0
TOTAL SHOTS: 694


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0041400406,10,201939,Stephen Curry,1610612744,Golden State Warriors,1,11,14,...,Center(C),16-24 ft.,20.0,-27.0,208.0,1,0,20150616,CLE,GSW
1,Shot Chart Detail,0041400406,25,201939,Stephen Curry,1610612744,Golden State Warriors,1,8,43,...,Center(C),Less Than 8 ft.,0.0,2.0,1.0,1,1,20150616,CLE,GSW
2,Shot Chart Detail,0041400406,27,201939,Stephen Curry,1610612744,Golden State Warriors,1,8,9,...,Left Side Center(LC),24+ ft.,25.0,-179.0,187.0,1,0,20150616,CLE,GSW
3,Shot Chart Detail,0041400406,50,201939,Stephen Curry,1610612744,Golden State Warriors,1,6,0,...,Center(C),Less Than 8 ft.,0.0,9.0,-3.0,1,1,20150616,CLE,GSW
4,Shot Chart Detail,0041400406,56,201939,Stephen Curry,1610612744,Golden State Warriors,1,5,15,...,Left Side(L),24+ ft.,22.0,-228.0,-3.0,1,1,20150616,CLE,GSW


# Giannis Antetokounmpo - Milwaukee Bucks (2020)

In [8]:
shots_df, finals_summary = finals_shots_last_10_seasons("Giannis Antetokounmpo", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2014-15                   0
1  2015-16                   0
2  2016-17                   0
3  2017-18                   0
4  2018-19                   0
5  2019-20                   0
6  2020-21                   6
7  2021-22                   0
8  2022-23                   0
9  2023-24                   0
TOTAL SHOTS: 123


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0042000406,15,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,1,11,6,...,Center(C),Less Than 8 ft.,3,27,24,1,1,20210720,MIL,PHX
1,Shot Chart Detail,0042000406,67,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,1,5,57,...,Left Side Center(LC),24+ ft.,27,-181,211,1,0,20210720,MIL,PHX
2,Shot Chart Detail,0042000406,137,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,1,1,42,...,Center(C),Less Than 8 ft.,2,-14,17,1,1,20210720,MIL,PHX
3,Shot Chart Detail,0042000406,140,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,1,1,13,...,Center(C),Less Than 8 ft.,1,6,12,1,1,20210720,MIL,PHX
4,Shot Chart Detail,0042000406,149,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,1,0,13,...,Center(C),Less Than 8 ft.,3,-29,14,1,0,20210720,MIL,PHX


# Nikola Jokic - Denver Nuggets (2023)

In [9]:
shots_df, finals_summary = finals_shots_last_10_seasons("Nikola Jokic", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2014-15                   0
1  2015-16                   0
2  2016-17                   0
3  2017-18                   0
4  2018-19                   0
5  2019-20                   0
6  2020-21                   0
7  2021-22                   0
8  2022-23                   5
9  2023-24                   0
TOTAL SHOTS: 96


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0042200405,83,203999,Nikola Jokić,1610612743,Denver Nuggets,1,5,28,...,Right Side Center(RC),24+ ft.,25,86,243,1,1,20230612,DEN,MIA
1,Shot Chart Detail,0042200405,237,203999,Nikola Jokić,1610612743,Denver Nuggets,2,8,4,...,Center(C),Less Than 8 ft.,5,-5,50,1,1,20230612,DEN,MIA
2,Shot Chart Detail,0042200405,258,203999,Nikola Jokić,1610612743,Denver Nuggets,2,6,56,...,Center(C),Less Than 8 ft.,2,-16,22,1,1,20230612,DEN,MIA
3,Shot Chart Detail,0042200405,274,203999,Nikola Jokić,1610612743,Denver Nuggets,2,5,0,...,Center(C),Less Than 8 ft.,7,-43,62,1,1,20230612,DEN,MIA
4,Shot Chart Detail,0042200405,295,203999,Nikola Jokić,1610612743,Denver Nuggets,2,2,44,...,Center(C),24+ ft.,26,-11,260,1,0,20230612,DEN,MIA


# Jimmy Butler - Miami Heat (2020, 2023)

In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("Jimmy Butler", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2015-16                   0
1  2016-17                   0
2  2017-18                   0
3  2018-19                   0
4  2019-20                   6
5  2020-21                   0
6  2021-22                   0
7  2022-23                   5
8  2023-24                   0
TOTAL SHOTS: 188


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0041900406,49,202710,Jimmy Butler III,1610612748,Miami Heat,1,7,56,...,Center(C),Less Than 8 ft.,6,-7,62,1,0,20201011,MIA,LAL
1,Shot Chart Detail,0041900406,107,202710,Jimmy Butler III,1610612748,Miami Heat,1,2,31,...,Left Side(L),24+ ft.,24,-234,79,1,0,20201011,MIA,LAL
2,Shot Chart Detail,0041900406,115,202710,Jimmy Butler III,1610612748,Miami Heat,1,1,38,...,Right Side(R),24+ ft.,22,226,8,1,1,20201011,MIA,LAL
3,Shot Chart Detail,0041900406,177,202710,Jimmy Butler III,1610612748,Miami Heat,2,10,16,...,Center(C),Less Than 8 ft.,0,0,-6,1,0,20201011,MIA,LAL
4,Shot Chart Detail,0041900406,212,202710,Jimmy Butler III,1610612748,Miami Heat,2,7,57,...,Center(C),8-16 ft.,12,18,120,1,1,20201011,MIA,LAL


# High-Usage Shooters 
### (Klay Thompson, Jayson Tatum, Devin Booker, Jamal Murray)

# Klay Thompson - Golden State Warriors (2015, 2016, 2017, 2018, 2019, 2022)

In [10]:
shots_df, finals_summary = finals_shots_last_10_seasons("Klay Thompson", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2014-15                   6
1  2015-16                   7
2  2016-17                   5
3  2017-18                   4
4  2018-19                   5
5  2019-20                   0
6  2020-21                   0
7  2021-22                   6
8  2022-23                   0
9  2023-24                   0
TOTAL SHOTS: 504


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0041400406,14,202691,Klay Thompson,1610612744,Golden State Warriors,1,10,25,...,Left Side Center(LC),16-24 ft.,17.0,-120.0,132.0,1,1,20150616,CLE,GSW
1,Shot Chart Detail,0041400406,116,202691,Klay Thompson,1610612744,Golden State Warriors,2,11,33,...,Left Side(L),8-16 ft.,14.0,-89.0,112.0,1,0,20150616,CLE,GSW
2,Shot Chart Detail,0041400406,127,202691,Klay Thompson,1610612744,Golden State Warriors,2,10,19,...,Left Side Center(LC),24+ ft.,25.0,-224.0,123.0,1,0,20150616,CLE,GSW
3,Shot Chart Detail,0041400406,420,202691,Klay Thompson,1610612744,Golden State Warriors,3,0,37,...,Left Side Center(LC),24+ ft.,24.0,-168.0,179.0,1,0,20150616,CLE,GSW
4,Shot Chart Detail,0041400406,453,202691,Klay Thompson,1610612744,Golden State Warriors,4,10,39,...,Center(C),8-16 ft.,11.0,-8.0,113.0,1,0,20150616,CLE,GSW


# Jayson Tatum

In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("Jayson Tatum", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2015-16                   0
1  2016-17                   0
2  2017-18                   0
3  2018-19                   0
4  2019-20                   0
5  2020-21                   0
6  2021-22                   6
7  2022-23                   0
8  2023-24                   5
TOTAL SHOTS: 218


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0042100406,20,1628369,Jayson Tatum,1610612738,Boston Celtics,1,10,34,...,Center(C),Less Than 8 ft.,4,-45,21,1,0,20220616,BOS,GSW
1,Shot Chart Detail,0042100406,31,1628369,Jayson Tatum,1610612738,Boston Celtics,1,9,27,...,Left Side Center(LC),24+ ft.,25,-130,217,1,1,20220616,BOS,GSW
2,Shot Chart Detail,0042100406,53,1628369,Jayson Tatum,1610612738,Boston Celtics,1,8,3,...,Left Side Center(LC),16-24 ft.,17,-93,154,1,1,20220616,BOS,GSW
3,Shot Chart Detail,0042100406,66,1628369,Jayson Tatum,1610612738,Boston Celtics,1,6,27,...,Left Side Center(LC),24+ ft.,25,-92,243,1,0,20220616,BOS,GSW
4,Shot Chart Detail,0042100406,120,1628369,Jayson Tatum,1610612738,Boston Celtics,1,2,53,...,Left Side Center(LC),16-24 ft.,16,-61,152,1,1,20220616,BOS,GSW


# Devin Booker

In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("Devin Booker", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2015-16                   0
1  2016-17                   0
2  2017-18                   0
3  2018-19                   0
4  2019-20                   0
5  2020-21                   6
6  2021-22                   0
7  2022-23                   0
8  2023-24                   0
TOTAL SHOTS: 143


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0042000406,17,1626164,Devin Booker,1610612756,Phoenix Suns,1,10,48,...,Right Side(R),8-16 ft.,14,84,121,1,1,20210720,MIL,PHX
1,Shot Chart Detail,0042000406,21,1626164,Devin Booker,1610612756,Phoenix Suns,1,10,13,...,Right Side Center(RC),24+ ft.,26,117,239,1,0,20210720,MIL,PHX
2,Shot Chart Detail,0042000406,50,1626164,Devin Booker,1610612756,Phoenix Suns,1,8,1,...,Left Side(L),8-16 ft.,12,-123,17,1,0,20210720,MIL,PHX
3,Shot Chart Detail,0042000406,108,1626164,Devin Booker,1610612756,Phoenix Suns,1,3,22,...,Center(C),Less Than 8 ft.,3,22,29,1,0,20210720,MIL,PHX
4,Shot Chart Detail,0042000406,123,1626164,Devin Booker,1610612756,Phoenix Suns,1,2,50,...,Center(C),24+ ft.,26,57,260,1,0,20210720,MIL,PHX


# Jamal Murray - Denver Nuggets (2023)

In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("Jamal Murray", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2015-16                   0
1  2016-17                   0
2  2017-18                   0
3  2018-19                   0
4  2019-20                   0
5  2020-21                   0
6  2021-22                   0
7  2022-23                   5
8  2023-24                   0
TOTAL SHOTS: 91


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0042200405,50,1627750,Jamal Murray,1610612743,Denver Nuggets,1,7,15,...,Center(C),24+ ft.,25,56,253,1,0,20230612,DEN,MIA
1,Shot Chart Detail,0042200405,65,1627750,Jamal Murray,1610612743,Denver Nuggets,1,6,47,...,Center(C),Less Than 8 ft.,1,1,16,1,1,20230612,DEN,MIA
2,Shot Chart Detail,0042200405,146,1627750,Jamal Murray,1610612743,Denver Nuggets,1,1,34,...,Left Side Center(LC),24+ ft.,25,-137,220,1,0,20230612,DEN,MIA
3,Shot Chart Detail,0042200405,158,1627750,Jamal Murray,1610612743,Denver Nuggets,1,1,11,...,Left Side Center(LC),24+ ft.,25,-170,196,1,0,20230612,DEN,MIA
4,Shot Chart Detail,0042200405,171,1627750,Jamal Murray,1610612743,Denver Nuggets,1,0,1,...,Left Side Center(LC),16-24 ft.,20,-137,149,1,0,20230612,DEN,MIA


# Low-Usage Role Players
### (Bruce Brown, Kevon Looney, Aaron Gordon, Derrick White)

# Bruce Brown - Nuggets (2023)

In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("Bruce Brown", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2015-16                   0
1  2016-17                   0
2  2017-18                   0
3  2018-19                   0
4  2019-20                   0
5  2020-21                   0
6  2021-22                   0
7  2022-23                   5
8  2023-24                   0
TOTAL SHOTS: 46


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0042200405,106,1628971,Bruce Brown,1610612743,Denver Nuggets,1,3,53,...,Left Side(L),8-16 ft.,8,-44,67,1,0,20230612,DEN,MIA
1,Shot Chart Detail,0042200405,132,1628971,Bruce Brown,1610612743,Denver Nuggets,1,2,30,...,Left Side(L),24+ ft.,23,-229,25,1,0,20230612,DEN,MIA
2,Shot Chart Detail,0042200405,154,1628971,Bruce Brown,1610612743,Denver Nuggets,1,1,16,...,Center(C),Less Than 8 ft.,5,-34,47,1,0,20230612,DEN,MIA
3,Shot Chart Detail,0042200405,196,1628971,Bruce Brown,1610612743,Denver Nuggets,2,10,50,...,Center(C),Less Than 8 ft.,2,20,8,1,0,20230612,DEN,MIA
4,Shot Chart Detail,0042200405,218,1628971,Bruce Brown,1610612743,Denver Nuggets,2,9,32,...,Center(C),Less Than 8 ft.,2,26,7,1,1,20230612,DEN,MIA
